<a href="https://colab.research.google.com/github/kQukka/SoDA_Mat_Agile/blob/main/5.yt_kim/qlearning_%EC%A0%95%EB%A6%AC_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#모듈

In [17]:
#코랩은 gym버전이 낮아서 업그레이드 해야 reset 에러가 발생하지 않음
!pip install --upgrade gym

In [18]:
!pip install pygame

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import random as pr

from contextlib import closing
from io import StringIO
import os
from os import path
from typing import Optional

import pygame
from pygame.constants import SRCALPHA

import gym
from gym import Env, spaces, utils
#from gym.envs.toy_text.utils import categorical_sample
from gym.envs.registration import register

In [20]:
#categorical_sample 모듈 임포트 에러
# 링크 : https://github.com/openai/gym/blob/master/gym/envs/toy_text/utils.py
def categorical_sample(prob_n, np_random):
    """
    Sample from categorical distribution
    Each row specifies class probabilities
    """
    # np.asarray는 array를 복사할 때 사용한다.
    # 무조건 복사하지는 않고 데이터의 형태가 다를 경우에만 복사한다
    # 참고 : https://supermemi.tistory.com/66
    prob_n = np.asarray(prob_n)

    #누적합을 구할 때 np.cumsum을 사용
    csprob_n = np.cumsum(prob_n)
    return (csprob_n > np_random.random()).argmax()

#기본설정값

In [21]:
#common.setting.py
NUM_EPISODES = 200000
NUM_ROW = 10
NUM_COL = 9

POINT_ITEM = {'A': (5, 0), 'B': (4, 0), 'C': (3, 0), 'D': (2, 0),
              'E': (0, 0), 'F': (0, 1), 'G': (0, 2), 'H': (0, 3), 'I': (0, 4),
              'J': (0, 5), 'K': (0, 6), 'L': (0, 7), 'M': (0, 8),
              'N': (2, 8), 'O': (3, 8), 'P': (4, 8), 'Q': (5, 8)}

POINT_START = (9, 4)
POINT_END = (9, 4)

POINT_WALL = [(3, 2), (3, 4), (3, 6),
              (4, 2), (4, 4), (4, 6),
              (5, 2), (5, 4), (5, 6),
              (6, 2), (6, 4), (6, 6),
              (9, 0), (9, 1), (9, 2), (9, 3),
              (9, 5), (9, 6), (9, 7), (9, 8)]

#logstic_env.common.py
IDX_ACTION_LEFT = 0
IDX_ACTION_DOWN = 1
IDX_ACTION_RIGHT = 2
IDX_ACTION_UP = 3
IDX_ACTION = [IDX_ACTION_LEFT, IDX_ACTION_DOWN, IDX_ACTION_RIGHT, IDX_ACTION_UP]

INITIAL_ACTION_UP = 'U'
INITIAL_ACTION_DOWN = 'D'
INITIAL_ACTION_RIGHT = 'R'
INITIAL_ACTION_LEFT = 'L'
INITIAL_ACTION = [INITIAL_ACTION_LEFT, INITIAL_ACTION_DOWN, INITIAL_ACTION_RIGHT, INITIAL_ACTION_UP]

STR_ACTION_UP = 'Up'
STR_ACTION_DOWN = 'Down'
STR_ACTION_RIGHT = 'Right'
STR_ACTION_LEFT = 'Left'
STR_ACTION = [STR_ACTION_UP, STR_ACTION_DOWN, STR_ACTION_RIGHT, STR_ACTION_LEFT]

INITIAL_ENV_START = 'S'
INITIAL_ENV_GOAL = 'G'
INITIAL_ENV_FLOOR = 'F'
INITIAL_ENV_WALL = 'H'


#logistic_env.env_gym.py
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3

MAPS = {
    "4x4": ["SFFF", "FHFH", "FFFH", "HFFG"],
    "8x8": [
        "SFFFFFFF",
        "FFFFFFFF",
        "FFFHFFFF",
        "FFFFFHFF",
        "FFFHFFFF",
        "FHHFFFHF",
        "FHFFHFHF",
        "FFFHFFFG",
    ],
}


IDX_LOG_STATE = 0
IDX_LOG_Q_MAP = 1
IDX_LOG_REWARD = 2
IDX_LOG_ACTION = 3

#csv save, load함수 + random_argmax 함수

In [22]:
#common.csv_.py
import csv

def save(path, str_):
    with open(path, 'w', encoding='utf-8', newline='') as f:
        wr = csv.writer(f)
        for s in str_:
            wr.writerow(s)

def load(path):
    with open(path, 'r', encoding='utf-8', newline='') as f:
        rd = csv.reader(f)
        data = []
        for s in rd:
            data.append([float(v) for v in s])
    return data


#common.func.py
def random_argmax(vector):
    # np.amax : 최댓값 반환
    m = np.amax(vector)
    #0이 아닌 최댓값 m을 반환
    indices = np.nonzero(vector == m)[0]
    # pr.choice : random choice
    # indices에서 하나의 값을 랜덤으로 리턴
    return pr.choice(indices)

# get_set_env 함수

In [23]:
#환경설정값 => input(집어와야하는아이템목록), 시작지점, 종료지점, 벽, 아이템 위치 설정
def get_set_env(input_, p_start, p_end, p_wall, p_item):
    #p_input = [(9,4)] + [(3,0),(0,5)] +[(9,4)]
    #위의값은 C,J가 인풋으로 들어왔을 경우, (9,4) = 시작점, 도착점
    p_input = [p_start] + [p_item[ch] for ch in input_] + [p_end]

    p_start_end = []
    for idx in range(len(p_input)-1):
        p_start_end.append([p_input[idx], p_input[idx+1]])

    #print(p_start_end)
    # >>> [[(9, 4), (3, 0)], [(3, 0), (0, 5)], [(0, 5), (9, 4)]]

    buf_wall = []
    for p_s, p_e in p_start_end:
        
        #print(p_s, p_e)
        #(9, 4) (3, 0)
        #(3, 0) (0, 5)
        #(0, 5) (9, 4)

        buf_p = []
        if p_s not in [p_start, p_end]:
            buf_p.append(p_s)
        if p_e not in [p_start, p_end]:
            buf_p.append(p_e)

        for p_ in p_item.values():
            if (p_ == p_s) or (p_ == p_e):
                continue
            buf_p.append(p_)

        # 자료가 리스트형일 떄
        # append는 리스트에 리스트 형태의 자료를 그대로 넣어서 2차원으로 만들고
        # extend는 기존 리스트에 값을 추가하여 결과적으로 1차원이 되도록 만든다
        buf_p.extend(p_wall)
        buf_wall.append(buf_p)
    return zip(p_start_end, buf_wall)

#메인함수1

In [24]:
input_ = ["C", "J"]

# 환경설정값에 input, 시작점, 도착점, 벽, 아이템을 입력
p_setting_env = list(get_set_env(input_, POINT_START, POINT_END, POINT_WALL, POINT_ITEM))

id_env = ['LogisEnv-v1', 'LogisEnv-v2', 'LogisEnv-v3']
path_map = ['./logistic_q_learning_map_1.csv', './logistic_q_learning_map_2.csv', './logistic_q_learning_map_3.csv']
path = ['./logistic_q_learning_1_1.csv', './logistic_q_learning_1_2.csv', './logistic_q_learning_1_3.csv']

list_q_map = []
list_agent = []

#create_map, print_env_map, generate_random_map 함수

In [25]:
def create_map(num_row, num_col, start=None, goal=None, wall=None):
    if start is None:
        start = [0, 0]
    if goal is None:
        goal = [num_row, num_col]
    map_ = [[INITIAL_ENV_FLOOR for _ in range(num_col)] for _ in range(num_row)]

    #환경에서 벽을 만드는 부분
    for idx_row, idx_col in wall:
        map_[idx_row][idx_col] = INITIAL_ENV_WALL

    #시작지점, 종료지점 지정
    map_[start[0]][start[1]] = INITIAL_ENV_START
    map_[goal[0]][goal[1]] = INITIAL_ENV_GOAL
    return ["".join(x) for x in map_]

#맵출력용 함수
def print_env_map(env_map):
    for map_ in env_map:
        print(map_)


def generate_random_map(size=8, p=0.8):
    """Generates a random valid map (one that has a path from start to goal)
    시작점에서 도착점까지의 길을 가지는 랜덤한 맵을 만든다
    :param size: size of each side of the grid
    param size : 각 그리드별 사이즈
    :param p: probability that a tile is frozen
    param p : 타일이 얼어있는지에 대한 가능성
    """
    valid = False

    # DFS to check that it's a valid path.
    # DFS(깊이 우선 탐색)알고리즘으로 유효한 길인지 체크
    def is_valid(res):
        frontier, discovered = [], set()
        frontier.append((0, 0))

        # 반복문을 반복하면서 frontier에 좌표정보 추가
        # discovered는 중복되지 않은 새로운 좌표만을 추가
        while frontier:
            r, c = frontier.pop()
            #첫 r c 의 값은 0 0, 이후 이동방향에 따라 r,c의 값이 달라짐
            if not (r, c) in discovered:
                discovered.add((r, c))
                #이동방향
                directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]
                for x, y in directions:
                    r_new = r + x
                    c_new = c + y
                    if r_new < 0 or r_new >= size or c_new < 0 or c_new >= size:
                        continue
                    if res[r_new][c_new] == "G":
                        return True
                    if res[r_new][c_new] != "H":
                        frontier.append((r_new, c_new))
        return False

    #S = start, G = Goal, F = Frozen, H = Hole
    while not valid:
        p = min(1, p)
        res = np.random.choice(["F", "H"], (size, size), p=[p, 1 - p])
        res[0][0] = "S"
        res[-1][-1] = "G"
        valid = is_valid(res)
    return ["".join(x) for x in res]

#LogisticEnv 클래스

In [26]:
class LogisticEnv(Env):
    """
    Logistic involves crossing a logistic map from Start(S) to Goal(G) without falling into any Holes(H) by walking over the Frozen(F) map.
    로지스틱은 얼어있는 맵을 걸으며 어떤 홀에도 빠지지 않고 시작지점부터 도착지점까지 건너는 것입니다.
    The agent may not always move in the intended direction due to the slippery nature of the logistic map.
    에이전트는 항상 의도하지 않은 방향으로 움직일 수 있습니다.

    ### Action Space
    The agent takes a 1-element vector for actions.
    The action space is `(dir)`, where `dir` decides direction to move in which can be:
    - 0: LEFT
    - 1: DOWN
    - 2: RIGHT
    - 3: UP

    ### Observation Space
    The observation is a value representing the agent's current position as current_row * nrows + current_col (where both the row and col start at 0).
    observation은 현재 위치를 current_row * nrows + current_col 로 나타낸 값입니다(0,0에서 시작)
    For example, the goal position in the 4x4 map can be calculated as follows: 3 * 4 + 3 = 15.
    The number of possible observations is dependent on the size of the map.
    가능한 observations의 수는 맵의 크기에 따라서 달라질 수 있습니다.
    For example, the 4x4 map has 16 possible observations.
    예를 들어, 4x4맵은 16개의 observations가 가능합니다.
    ### Rewards
    Reward schedule:
    - Reach goal(G): +1
    - Reach hole(H): 0
    - Reach frozen(F): 0
    ### Arguments
    ```
    gym.make('FrozenLake-v1', desc=None,map_name="4x4", is_slippery=True)
    ```
    `desc`: Used to specify custom map for logistic map. For example,
        desc=["SFFF", "FHFH", "FFFH", "HFFG"].
    `map_name`: ID to use any of the preloaded maps.
        "4x4":[
            "SFFF",
            "FHFH",
            "FFFH",
            "HFFG"
            ]
        "8x8": [
            "SFFFFFFF",
            "FFFFFFFF",
            "FFFHFFFF",
            "FFFFFHFF",
            "FFFHFFFF",
            "FHHFFFHF",
            "FHFFHFHF",
            "FFFHFFFG",
        ]
    `is_slippery`: True/False.
                   If True will move in intended direction with probability of 1/3 else will move in either perpendicular direction with equal probability of 1/3 in both directions.
                   만약 의도한 방향으로 움직일 가능성이 1/3이라면 1/3의 확률로 수직으로 이동할 것입니다.
        For example, if action is left and is_slippery is True, then:
        - P(move left)=1/3
        - P(move up)=1/3
        - P(move down)=1/3
    ### Version History
    * v1: Bug fixes to rewards
    * v0: Initial versions release (1.0.0)
    """

    metadata = {"render_modes": ["human", "ansi", "rgb_array"], "render_fps": 4}

    def __init__(self, map_name="4x4", map_env=None, random_map=None, is_slippery=True):
        # 인자에 따라 맵을 그대로 쓸 것인지, 랜덤맵을 사용할 것인지 정한다
        # MAPS에는 "4x4" "8x8" 두가지가 위에 설정되어있다.
        if map_env:
            desc = map_env
        elif random_map:
            desc = generate_random_map(random_map[0], random_map[1])
        else:
            desc = MAPS[map_name]

        self.desc = desc = np.asarray(desc, dtype="c")
        self.nrow, self.ncol = nrow, ncol = desc.shape
        self.reward_range = (0, 1)

        nA = 4
        nS = nrow * ncol

        self.initial_state_distrib = np.array(desc == b"S").astype("float64").ravel()
        self.initial_state_distrib /= self.initial_state_distrib.sum()

        #{0: {0: [], 1: [], 2: [], 3: []},       1: {0: [], 1: [], 2: [], 3: []}, ..... 형태
        self.P = {s: {a: [] for a in range(nA)} for s in range(nS)}

        def to_s(row, col):
            return row * ncol + col

        def inc(row, col, a):

            # 행동이 왼쪽일때 열을 하나뺀다
            # -1이 나올때에 대한 대비
            if a == LEFT:
                col = max(col - 1, 0)

            # 행동이 아래쪽일때 행을 하나 더하고, 행의 수를 하나 제거한다
            # row가 row의 최대 길이보다 더 커지는 것을 방지
            elif a == DOWN:
                row = min(row + 1, nrow - 1)
            elif a == RIGHT:
                col = min(col + 1, ncol - 1)
            elif a == UP:
                row = max(row - 1, 0)
            return (row, col)

        def update_probability_matrix(row, col, action):
            newrow, newcol = inc(row, col, action)
            newstate = to_s(newrow, newcol)
            newletter = desc[newrow, newcol]
            done = bytes(newletter) in b"GH"
            reward = 0
            if newletter == b"H":
                reward = -1
            elif newletter == b"G":
                reward = 1
            # reward = float(newletter == b"G")
            return newstate, reward, done

        for row in range(nrow):
            for col in range(ncol):
                s = to_s(row, col)
                for a in range(4):
                    li = self.P[s][a]
                    letter = desc[row, col]
                    if letter in b"GH":
                        li.append((1.0, s, 0, True))
                    else:
                        if is_slippery:
                            for b in [(a - 1) % 4, a, (a + 1) % 4]:
                                li.append(
                                    (1.0 / 3.0, *update_probability_matrix(row, col, b))
                                )
                        else:
                            li.append((1.0, *update_probability_matrix(row, col, a)))

        self.observation_space = spaces.Discrete(nS)
        self.action_space = spaces.Discrete(nA)

        # pygame utils
        self.window_size = (min(64 * ncol, 512), min(64 * nrow, 512))
        self.window_surface = None
        self.clock = None
        self.hole_img = None
        self.cracked_hole_img = None
        self.ice_img = None
        self.elf_images = None
        self.goal_img = None
        self.start_img = None


    def step(self, a):
        transitions = self.P[self.s][a]
        i = categorical_sample([t[0] for t in transitions], self.np_random)
        p, s, r, d = transitions[i]
        self.s = s
        self.lastaction = a
        return (int(s), r, d, {"prob": p})

    def reset(
            self,
            *,
            seed: Optional[int] = None,
            return_info: bool = False,
            options: Optional[dict] = None):
        super().reset(seed=seed)
        self.s = categorical_sample(self.initial_state_distrib, self.np_random)
        self.lastaction = None

        if not return_info:
            return int(self.s)
        else:
            return int(self.s), {"prob": 1}

    def render(self, mode="human"):
        desc = self.desc.tolist()
        if mode == "ansi":
            return self._render_text(desc)
        else:
            return self._render_gui(desc, mode)

    def _render_gui(self, desc, mode):
        if self.window_surface is None:
            pygame.init()
            pygame.display.init()
            #타이틀바의 텍스트 설정
            pygame.display.set_caption("Frozen Lake")
            if mode == "human":
                self.window_surface = pygame.display.set_mode(self.window_size)
            else:  # rgb_array
                self.window_surface = pygame.Surface(self.window_size)
        if self.clock is None:
            self.clock = pygame.time.Clock()
        if self.hole_img is None:
            file_name = path.join(path.dirname(__file__), "img/hole.png")
            self.hole_img = pygame.image.load(file_name)
        if self.cracked_hole_img is None:
            file_name = path.join(path.dirname(__file__), "img/cracked_hole.png")
            self.cracked_hole_img = pygame.image.load(file_name)
        if self.ice_img is None:
            file_name = path.join(path.dirname(__file__), "img/ice.png")
            self.ice_img = pygame.image.load(file_name)
        if self.goal_img is None:
            file_name = path.join(path.dirname(__file__), "img/goal.png")
            self.goal_img = pygame.image.load(file_name)
        if self.start_img is None:
            file_name = path.join(path.dirname(__file__), "img/stool.png")
            self.start_img = pygame.image.load(file_name)
        if self.elf_images is None:
            elfs = [
                path.join(path.dirname(__file__), "img/elf_left.png"),
                path.join(path.dirname(__file__), "img/elf_down.png"),
                path.join(path.dirname(__file__), "img/elf_right.png"),
                path.join(path.dirname(__file__), "img/elf_up.png"),
            ]
            self.elf_images = [pygame.image.load(f_name) for f_name in elfs]

        #surface생성, furface = 2d 객체를 의미
        board = pygame.Surface(self.window_size, flags=SRCALPHA)
        cell_width = self.window_size[0] // self.ncol
        cell_height = self.window_size[1] // self.nrow
        smaller_cell_scale = 0.6
        small_cell_w = smaller_cell_scale * cell_width
        small_cell_h = smaller_cell_scale * cell_height

        # prepare images
        last_action = self.lastaction if self.lastaction is not None else 1
        elf_img = self.elf_images[last_action]
        elf_scale = min(
            small_cell_w / elf_img.get_width(),
            small_cell_h / elf_img.get_height(),
        )
        elf_dims = (
            elf_img.get_width() * elf_scale,
            elf_img.get_height() * elf_scale,
        )
        elf_img = pygame.transform.scale(elf_img, elf_dims)
        hole_img = pygame.transform.scale(self.hole_img, (cell_width, cell_height))
        cracked_hole_img = pygame.transform.scale(
            self.cracked_hole_img, (cell_width, cell_height)
        )
        ice_img = pygame.transform.scale(self.ice_img, (cell_width, cell_height))
        goal_img = pygame.transform.scale(self.goal_img, (cell_width, cell_height))
        start_img = pygame.transform.scale(self.start_img, (small_cell_w, small_cell_h))

        for y in range(self.nrow):
            for x in range(self.ncol):
                rect = (x * cell_width, y * cell_height, cell_width, cell_height)
                if desc[y][x] == b"H":
                    self.window_surface.blit(hole_img, (rect[0], rect[1]))
                elif desc[y][x] == b"G":
                    self.window_surface.blit(ice_img, (rect[0], rect[1]))
                    goal_rect = self._center_small_rect(rect, goal_img.get_size())
                    self.window_surface.blit(goal_img, goal_rect)
                elif desc[y][x] == b"S":
                    self.window_surface.blit(ice_img, (rect[0], rect[1]))
                    stool_rect = self._center_small_rect(rect, start_img.get_size())
                    self.window_surface.blit(start_img, stool_rect)
                else:
                    self.window_surface.blit(ice_img, (rect[0], rect[1]))

                pygame.draw.rect(board, (180, 200, 230), rect, 1)

        # paint the elf
        bot_row, bot_col = self.s // self.ncol, self.s % self.ncol
        cell_rect = (
            bot_col * cell_width,
            bot_row * cell_height,
            cell_width,
            cell_height,
        )
        if desc[bot_row][bot_col] == b"H":
            self.window_surface.blit(cracked_hole_img, (cell_rect[0], cell_rect[1]))
        else:
            elf_rect = self._center_small_rect(cell_rect, elf_img.get_size())
            self.window_surface.blit(elf_img, elf_rect)

        self.window_surface.blit(board, board.get_rect())
        if mode == "human":
            # pygame 이벤트 핸들러 내부 처리
            pygame.event.pump()
            pygame.display.update()
            self.clock.tick(self.metadata["render_fps"])
        else:  # rgb_array
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(self.window_surface)), axes=(1, 0, 2)
            )

    @staticmethod
    def _center_small_rect(big_rect, small_dims):
        offset_w = (big_rect[2] - small_dims[0]) / 2
        offset_h = (big_rect[3] - small_dims[1]) / 2
        return (
            big_rect[0] + offset_w,
            big_rect[1] + offset_h,
        )

    def _render_text(self, desc):
        outfile = StringIO()

        row, col = self.s // self.ncol, self.s % self.ncol
        desc = [[c.decode("utf-8") for c in line] for line in desc]
        desc[row][col] = utils.colorize(desc[row][col], "red", highlight=True)
        if self.lastaction is not None:
            outfile.write(f"  ({['Left', 'Down', 'Right', 'Up'][self.lastaction]})\n")
        else:
            outfile.write("\n")
        outfile.write("\n".join("".join(line) for line in desc) + "\n")

        with closing(outfile):
            return outfile.getvalue()

# create_env 함수

In [27]:
def create_env(id_, manual=False, map_name='4x4', map_env=None, random_map=None, is_slippery=False):
    setting = None
    if manual:
        setting = {'map_env': map_env, 'is_slippery': is_slippery}
    elif random_map:
        setting = {'random_map': (random_map[0], random_map[1]), 'is_slippery': is_slippery}
    else:
        setting = {'map_name': map_name, 'is_slippery': is_slippery}

    register(
        id=id_,
        #LogisticEnv 클래스를 의미함
        entry_point=LogisticEnv,
        kwargs=setting
    )
    env = gym.make(id_)
    return env

# QLearning 클래스

In [28]:
class QLearning:
    def __init__(self, env):
        self.env = env
        # self.__log_epi = [[state_step, q_map_step, reward_step, act_step], ...]
        self.__log_epi = []

    # __get_log_item의 항목 참고
    def get_log_state(self):
        return self.__get_log_item(IDX_LOG_STATE)

    def get_log_q_map(self): 
        return self.__get_log_item(IDX_LOG_Q_MAP)

    def get_log_reward(self):
        return self.__get_log_item(IDX_LOG_REWARD)

    def get_log_action(self):
        return self.__get_log_item(IDX_LOG_ACTION)

    def save_q_map(self, path, q_map):
        return save(path, q_map)

    def load_q_map(self, path):
        return load(path)

    def run(self, num_episodes, q_map=None, early_stopping=False):
        del self.__log_epi
        self.__log_epi = []
        if early_stopping:
            early_stopping.clear()

        if q_map:
            q_map = np.array(q_map)

       # q_map이 지정되지 않은 경우, observation_space와 action_space 로 numpy.zeros의 행렬 만들기
        else:
            q_map = np.zeros([self.env.observation_space.n, self.env.action_space.n])

        num_progress = 0
        for i in range(num_episodes):
            # __run_episodes(self, q_map, idx=0, greedy=False, noise=False, learning_rate=0, discount=1):
            # self.__run_episodes(q_map)
            self.__run_episodes(q_map, noise=True, discount=0.9)

            #진행상황 나타낼 떄 사용
            num_ = int((i / num_episodes) * 100)
            if num_progress != num_:
                print(f'progress = {num_} %')
                num_progress = num_

            # 진행상황과 언제 early_stopping이 되었는지 표시할 때 사용한다.
            if early_stopping:
                if early_stopping.check_stopping(self.__log_epi[-1][IDX_LOG_ACTION]):
                    print(f'progress = {num_} %  --> {i}/{num_episodes} Early Stopping')
                    break
        sum_reward_by_epi = self.__get_log_sum_reward()
        return q_map, sum_reward_by_epi

    def __run_episodes(self, q_map, idx=0, greedy=False, noise=False, learning_rate=0, discount=1.):
        log_step = [[] for _ in range(4)]
        # 초기 state 설정 (Start)
        state = self.env.reset()

        done = False
        while not done:
            act = None

            # e-greedy
            if greedy:
                e = 1. / ((idx // 1000) + 1)
                if np.random.rand(1) < e:
                    act = self.env.action_space.sample()
                else:
                    act = random_argmax(q_map[state, :])
            # noise
            elif noise:
                act = np.argmax(q_map[state, :] + np.random.randn(1, self.env.action_space.n) / (idx + 1))
            else:
                act = random_argmax(q_map[state, :])
            new_state, reward, done, _ = self.env.step(act)

            if state != new_state:
                #러닝레이트가 존재하면 if, 존재하지 않을 경우 else를 실행한다 - 러닝레이트를 넣을 때와 넣지 않을때 식이 다르기때문
                if learning_rate > 0:
                    q_map[state, act] = (1 - learning_rate) * q_map[state, act] + learning_rate * (
                                reward + discount * np.max(q_map[new_state, :]))
                else:
                    q_map[state, act] = reward + discount * np.max(q_map[new_state, :])
            # log
            for idx, val in enumerate([state, 0, reward, act]):
                log_step[idx].append(val)
            # update state
            state = new_state
        self.__log_epi.append(log_step)
        return True

    def __get_log_sum_reward(self):
        return [sum(reward_epi) for reward_epi in self.get_log_reward()]

    # idx를 입력으로 받는다.(위의 경우에는 get_log_state부터 get_log_q_map..... 을 0, 1, 2, 3 순서로 받는다.)
    # __log_epi는 92번째 줄에서(현재줄로부터 7줄 위) log_step의 값을 받는다.
    # log_step은 [[], [], [], []]의 형태를 가지고, val은 [state, 0, reward, act] 의 값을 받아서 생성된다.
    # 최종적으로 for 문에 들어가는 __log_epi는 [[state, 0, reward, act], [state, 0, reward, act], [state, 0, reward, act], [state, 0, reward, act]] 의 형태를 가진다
    # log_epi[idx] 형태는 예를 들어 log_epi[0]일 경우 state, log_epi[1]일 경우 q_map을 의미하게 된다.
    def __get_log_item(self, idx):
        log_item = []
        for log_epi in self.__log_epi:
            log_item.append(log_epi[idx])
        return log_item

# earlystopping 클래스

In [29]:
class EarlyStopping:
    def __init__(self, patience=10, ratio=100):
        self.stack = []
        self.patience = patience
        self.ratio = ratio

    def clear(self):
        del self.stack
        self.stack = []

    # stack에 값을 추가하고 num을 check_ratio에서 세었던 num의 값으로 초기화한다
    # 위에서 입력한 비율보다 크거나 같을 경우 True, 아니면 False를 반환한다
    def check_stopping(self, value):
        self.__add_stack(value)
        num = self.__check_ratio()
        if ((num // self.patience)*100) >= self.ratio:
            return True
        return False

    #stack의 길이가 patience보다 클 경우 제일 앞 stack 제거
    def __add_stack(self, value):
        if len(self.stack) > self.patience:
            del self.stack[0]
        self.stack.append(value)

    # stack의 마지막값을 stand로 지정하고 num = 0으로 설정
    # stack의 마지막 값과 스택의 요소가 같으면 num에 +1을 더한다
    # 비율을 정하기 이전에 num을 세는 함수
    # stack은 1, 2, 3.... 과 같이 되어있을 수도 있지만 3, 3, 3, 3 .... 3 의 형식으로 되어있을 수도 있음
    # 앞의 경우는 num이 하나만 오를 수도 있지만, 뒤의 경우에는 num의 숫자가 계속 오름
    # early stoping의 조건을 충족시키기 위하여 사용
    def __check_ratio(self):
        stand = self.stack[-1]
        num = 0
        for val in self.stack:
            if stand == val:
                num += 1
        return num

#메인함수2

In [30]:
# p_start_end[0]에는 시작 좌표, p_start_end[1]에는 목표 좌표가 찍혀있음
for idx, (p_start_end, p_wall) in enumerate(p_setting_env):
    map_env = create_map(NUM_ROW, NUM_COL, start=p_start_end[0], goal=p_start_end[1], wall=p_wall)
    print_env_map(map_env)
    save(path_map[idx], map_env)

    env = create_env(id_=id_env[idx], manual=True, map_env=map_env)

    agent = QLearning(env)
    q_map, sum_reward_by_epi = agent.run(NUM_EPISODES, early_stopping=EarlyStopping())
    agent.save_q_map(path[idx], q_map.tolist())
    list_q_map.append(q_map)
    list_agent.append(agent)


path_ = []

HHHHHHHHH
FFFFFFFFF
HFFFFFFFH
GFHFHFHFH
HFHFHFHFH
HFHFHFHFH
FFHFHFHFF
FFFFFFFFF
FFFFFFFFF
HHHHSHHHH


/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:595: UserWarning: WARN: Overriding environment LogisEnv-v1
  logger.warn(f"Overriding environment {id}")
/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:506: UserWarning: WARN: The environment LogisEnv-v1 is out of date. You should consider upgrading to version `v3` with the environment ID `LogisEnv-v3`.
  f"The environment {path} is out of date. You should consider "


progress = 1 %
progress = 2 %
progress = 3 %
progress = 4 %
progress = 5 %
progress = 6 %
progress = 7 %
progress = 8 %
progress = 9 %
progress = 10 %
progress = 11 %
progress = 12 %
progress = 13 %
progress = 14 %
progress = 15 %
progress = 16 %
progress = 17 %
progress = 18 %
progress = 19 %
progress = 20 %
progress = 21 %
progress = 22 %
progress = 23 %
progress = 24 %
progress = 25 %
progress = 26 %
progress = 27 %
progress = 28 %
progress = 29 %
progress = 30 %
progress = 31 %
progress = 32 %
progress = 33 %
progress = 34 %
progress = 35 %
progress = 36 %
progress = 37 %
progress = 38 %
progress = 39 %
progress = 40 %
progress = 41 %
progress = 42 %
progress = 43 %
progress = 44 %
progress = 45 %
progress = 46 %
progress = 47 %
progress = 48 %
progress = 49 %
progress = 50 %
progress = 51 %
progress = 52 %
progress = 53 %
progress = 54 %
progress = 55 %
progress = 56 %
progress = 57 %
progress = 58 %
progress = 59 %
progress = 60 %
progress = 61 %
progress = 62 %
progress = 63 %
p

/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:595: UserWarning: WARN: Overriding environment LogisEnv-v2
  logger.warn(f"Overriding environment {id}")
/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:506: UserWarning: WARN: The environment LogisEnv-v2 is out of date. You should consider upgrading to version `v3` with the environment ID `LogisEnv-v3`.
  f"The environment {path} is out of date. You should consider "


progress = 1 %
progress = 2 %
progress = 3 %
progress = 4 %
progress = 5 %
progress = 6 %
progress = 7 %
progress = 8 %
progress = 9 %
progress = 10 %
progress = 11 %
progress = 12 %
progress = 13 %
progress = 14 %
progress = 15 %
progress = 16 %
progress = 17 %
progress = 18 %
progress = 19 %
progress = 20 %
progress = 21 %
progress = 22 %
progress = 23 %
progress = 24 %
progress = 25 %
progress = 26 %
progress = 27 %
progress = 28 %
progress = 29 %
progress = 30 %
progress = 31 %
progress = 32 %
progress = 33 %
progress = 34 %
progress = 35 %
progress = 36 %
progress = 37 %
progress = 38 %
progress = 39 %
progress = 40 %
progress = 41 %
progress = 42 %
progress = 43 %
progress = 44 %
progress = 45 %
progress = 46 %
progress = 47 %
progress = 48 %
progress = 49 %
progress = 50 %
progress = 51 %
progress = 52 %
progress = 53 %
progress = 54 %
progress = 55 %
progress = 56 %
progress = 57 %
progress = 58 %
progress = 59 %
progress = 60 %
progress = 61 %
progress = 62 %
progress = 63 %
p

/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:595: UserWarning: WARN: Overriding environment LogisEnv-v3
  logger.warn(f"Overriding environment {id}")


progress = 1 %
progress = 2 %
progress = 3 %
progress = 4 %
progress = 5 %
progress = 6 %
progress = 7 %
progress = 8 %
progress = 9 %
progress = 10 %
progress = 11 %
progress = 12 %
progress = 13 %
progress = 14 %
progress = 15 %
progress = 16 %
progress = 17 %
progress = 18 %
progress = 19 %
progress = 20 %
progress = 21 %
progress = 22 %
progress = 23 %
progress = 24 %
progress = 25 %
progress = 26 %
progress = 27 %
progress = 28 %
progress = 29 %
progress = 30 %
progress = 31 %
progress = 32 %
progress = 33 %
progress = 34 %
progress = 35 %
progress = 36 %
progress = 37 %
progress = 38 %
progress = 39 %
progress = 40 %
progress = 41 %
progress = 42 %
progress = 43 %
progress = 44 %
progress = 45 %
progress = 46 %
progress = 47 %
progress = 48 %
progress = 49 %
progress = 50 %
progress = 51 %
progress = 52 %
progress = 53 %
progress = 54 %
progress = 55 %
progress = 56 %
progress = 57 %
progress = 58 %
progress = 59 %
progress = 60 %
progress = 61 %
progress = 62 %
progress = 63 %
p

# get_idx_direct, get_move_p, get_path, print_str_direct 함수

In [31]:
def get_idx_direct(q_map):
    cnt = 0
    map_idx_direct = []
    while cnt < len(q_map)-1:
        for _ in range(9):
            #q_map에서 가장 큰 값을 val_max로 초기화한다
            val_max = q_map[cnt].max()
            # 가장 큰 값이 없으면 None을 추가하고
            if val_max == 0:
                map_idx_direct.append(None)
            # 있다면 q_map[cnt]에서 val_max의 index값을 idx로 초기화하고 map_idx_direct에 추가한다 
            # IDX_ACTION = [IDX_ACTION_LEFT, IDX_ACTION_DOWN, IDX_ACTION_RIGHT, IDX_ACTION_UP]
            else:
                idx = list(q_map[cnt]).index(val_max)
                if idx in IDX_ACTION:
                    map_idx_direct.append(idx)
            cnt += 1
    return map_idx_direct

# p_state = [row, col], move = list(0~3사이의 값들이 있음)
# 이동 방향에 따라 row, col 업데이트
def get_move_p(p_state, move, max_row, max_col):
    row = p_state[0]
    col = p_state[1]

    if move == IDX_ACTION_UP:
        row -= 1
    elif move == IDX_ACTION_DOWN:
        row += 1
    elif move == IDX_ACTION_LEFT:
        col -= 1
    elif move == IDX_ACTION_RIGHT:
        col += 1
    else:
        return False

    # row =  1번. 만약 row가 0보다 작으면 0
    #        2번. row>=max_row라면 max_row-1
    #        3번. 1번, 2번이 모두 아닐경우 row
    row = 0 if row < 0 else max_row-1 if row >= max_row else row
    col = 0 if col < 0 else max_col-1 if col >= max_col else col
    return [row, col]

#p_start, p_end = p_start_end[0], p_start_end[1]
def get_path(map_idx_direct, p_start, p_end, max_row, max_col):
    row = p_start[0]
    col = p_start[1]

    path_ = [[row, col]]
    # map_idx_direct = 0,1,2,3 의 방향을 가진다. 이동경로이므로 여러개의 값이 리스트 형태로 들어있음
    for _ in range(len(map_idx_direct)):

        # 2차원 배열처럼 사용
        # map_idx_direct는 1차원 리스트
        # row, col은 원래 2차원이므로 (row * max_col) + col을 할 경우 2차원배열을 1차원 배열로 바꾸었을 때 값이 출력될 수 있음
        # 예를들어 1차원 배열이 a = [0 1 2 3 4 5 6 7 8]이고, 
        # 2차원 배열이
        # 0 1 2
        # 3 4 5
        # 6 7 8
        # 일 때 숫자 5를 뽑고 싶으면 2차원배열에서는 (1,2)로 지정이 된다.
        # 즉 (row * max_col) = (1 * 2) = 2가 되고 현재 col인 2를 더하면 a[2+2]가 되어서 a[4]의 값인 5를 출력할 수 있다.
        idx_direct = map_idx_direct[(row*max_col)+col]

        # [row,col] = p_state
        # idx_direct는 하나의 방향이 나옴(0~3의 값)
        # get_move_p 로 (row,col)의 값을 업데이트 한다
        p_next = get_move_p([row, col], idx_direct, max_row, max_col)

        # p_next가 존재하지 않거나 이미 지정된 값(row, col)과 p_next가 같다면 반복문을 빠져나간다.
        if (p_next is False) or (p_next == [row, col]):
            break
        row = p_next[0]
        col = p_next[1]
        path_.append(p_next)
        if p_next == p_end:
            break
    return path_

# NUM_COL = 9로 설정되어있음
# idx_direct = map_idx_direct = 1차원 리스트
def print_str_direct(idx_direct, num_col):
    char_direct = []
    cnt = 0
    while cnt < len(idx_direct)-1:
        txt = ''
        buf = []
        for idx in range(num_col):
            char_ = None
            # IDX_ACTION = [IDX_ACTION_LEFT, IDX_ACTION_DOWN, IDX_ACTION_RIGHT, IDX_ACTION_UP]
            # IDX_ACTION_LEFT = 0
            # IDX_ACTION_DOWN = 1
            # IDX_ACTION_RIGHT = 2
            # IDX_ACTION_UP = 3
            if idx_direct[cnt] in IDX_ACTION:
                # INITIAL_ACTION = [INITIAL_ACTION_LEFT, INITIAL_ACTION_DOWN, INITIAL_ACTION_RIGHT, INITIAL_ACTION_UP]
                # INITIAL_ACTION_UP = 'U'
                # INITIAL_ACTION_DOWN = 'D'
                # INITIAL_ACTION_RIGHT = 'R'
                # INITIAL_ACTION_LEFT = 'L'

                char_ = INITIAL_ACTION[idx_direct[cnt]]
            else:
                char_ = ' '
            txt += char_
            buf.append(char_)

            # 출력시 칸 구분할 때 "ㅣ" 사용
            if idx < num_col-1:
                txt += ' | '
            cnt += 1
        print(txt)
        char_direct.append(buf)
    return char_direct

#메인함수3

In [32]:
for idx, (p_start_end, p_wall) in enumerate(p_setting_env):
    q_map = list_q_map[idx]
    map_idx_direct = get_idx_direct(q_map)
    buf_path = get_path(map_idx_direct, p_start_end[0], p_start_end[1], NUM_ROW, NUM_COL)
    path_.append(buf_path)
    print_str_direct(map_idx_direct, NUM_COL)

  |   |   |   |   |   |   |   |  
R | D | L | L | L | L | L | L | L
  | D | L | L | L | L | L | L |  
  | L |   | U |   | U |   | U |  
  | U |   | U |   | U |   | U |  
  | U |   | U |   | U |   | U |  
R | U |   | D |   | D |   | D | L
R | U | L | L | L | L | L | L | L
R | U | L | L | L | L | L | L | L
  |   |   |   | U |   |   |   |  
  |   |   |   |   |   |   |   |  
R | R | R | R | R | U | L | L | L
  | R | R | R | R | U | L | L |  
R | U |   | U |   | U |   | U |  
  | U |   | U |   | U |   | U |  
  | U |   | U |   | U |   | U |  
R | U |   | U |   | U |   | U | L
R | R | R | R | R | U | L | L | L
R | R | R | R | R | U | L | L | L
  |   |   |   | U |   |   |   |  
  |   |   |   |   | D |   |   |  
R | D | D | D | L | D | L | L | L
  | D | R | D | L | D | L | L |  
  | D |   | D |   | D |   | D |  
  | D |   | D |   | D |   | D |  
  | D |   | D |   | D |   | D |  
D | D |   | D |   | D |   | D | L
D | D | D | D | D | L | L | L | L
R | R | R | R | D | L | L | L | L
  |   |   |   